In [1]:
import sys
import requests
import pandas as pd 
import math

url_items_tp_data = (   'https://api.datawars2.ie/gw2/v1/items/json?fields='
                        'id,name,buy_price,sell_price,roi,profit,'
                        '7d_buy_listed,7d_buy_sold,7d_sell_listed,7d_sell_sold,'
                        '1d_buy_listed,1d_buy_sold,1d_sell_listed,1d_sell_sold')


result = requests.get(url_items_tp_data)

if(result.status_code == 200):
    data = result.json()
    tp_data = pd.json_normalize(data)

else:
	sys.exit("Status code not 200")

df = pd.read_json("database/refinement_recipes.json")
df = df.convert_dtypes()

output_item_price = []
crafting_profit = []
sell_velocity = []
est_crafting_profit = []

ACQUISITION_TYPE = 'buy_price' # order buy
# ACQUISITION_TYPE = 'sell_price' # instant buy
# DISPOSAL_TYPE = 'buy_price' # instant sell
DISPOSAL_TYPE = 'sell_price' # order sell

for i, row in df.iterrows():

    tp_ingredient_cost = 0
    vendor_ingredient_cost = 0
    output_item_count = row['output_item_count']
    output_item_price.append(int(tp_data[tp_data['name'] == row['output_item_name']][DISPOSAL_TYPE].values[0]))

    for ingredient in row['tp_ingredients']:
        ingredient['price'] = tp_data[tp_data['name'] == ingredient['ing_name']][ACQUISITION_TYPE].values[0]
        tp_ingredient_cost += ingredient['price'] * ingredient['count']

    for ingredient in row['vendor_ingredients']:
        vendor_ingredient_cost += ingredient['price'] * ingredient['count']

    # calculate refining profit
    crafting_price = (tp_ingredient_cost + vendor_ingredient_cost)/output_item_count
    sell_prize_minus_taxes = output_item_price[-1] - (output_item_price[-1] * 0.05) - (output_item_price[-1] * 0.1)
    crafting_profit.append(int(sell_prize_minus_taxes - crafting_price))

    # calculate selling daily velocity of 5% from last 7 days
    sell_velocity.append(int(tp_data[tp_data['name'] == row['output_item_name']]['7d_sell_sold'] / 7 * 0.05))

    # calculate estimate crafting profit (profit * sell_velocity)
    est_crafting_profit.append(int(crafting_profit[-1] * sell_velocity[-1]))



df['output_item_price'] = output_item_price
df['crafting_profit'] = crafting_profit
df['sell_velocity'] = sell_velocity
df['est_crafting_profit'] = est_crafting_profit


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
	display(df[[
        'output_item_name',
        'output_item_price',
        'crafting_profit',
        'est_crafting_profit',
        'sell_velocity',
        'tp_ingredients',
        'vendor_ingredients',
        'disciplines'
    ]]

	# .sort_values(by='crafting_profit', ascending=False)
	.sort_values(by='est_crafting_profit', ascending=False)

	.style.hide_index()
	)


# Chef: 441
# Armorsmith: 500
# Weaponsmith: 81
# Jeweler: 400


output_item_name,output_item_price,crafting_profit,est_crafting_profit,sell_velocity,tp_ingredients,vendor_ingredients,disciplines
Elder Wood Plank,233,12,246876,20573,"[{'ing_name': 'Elder Wood Log', 'count': 3, 'price': 62.0}]",[],"['Artificer', 'Huntsman', 'Scribe', 'Weaponsmith']"
Bolt of Silk,59,14,91672,6548,"[{'ing_name': 'Silk Scrap', 'count': 2, 'price': 18.0}]",[],"['Armorsmith', 'Leatherworker', 'Scribe', 'Tailor']"
Cured Thick Leather Square,373,13,49075,3775,"[{'ing_name': 'Thick Leather Section', 'count': 4, 'price': 76.0}]",[],"['Armorsmith', 'Artificer', 'Huntsman', 'Leatherworker', 'Scribe', 'Tailor', 'Weaponsmith']"
Mithril Ingot,65,3,45447,15149,"[{'ing_name': 'Mithril Ore', 'count': 2, 'price': 26.0}]",[],"['Armorsmith', 'Artificer', 'Huntsman', 'Jeweler', 'Leatherworker', 'Scribe', 'Tailor', 'Weaponsmith']"
Cured Thin Leather Square,149,12,32256,2688,"[{'ing_name': 'Thin Leather Section', 'count': 2, 'price': 57.0}]",[],"['Armorsmith', 'Artificer', 'Huntsman', 'Leatherworker', 'Scribe', 'Tailor', 'Weaponsmith']"
Silver Ingot,28,3,10902,3634,"[{'ing_name': 'Silver Ore', 'count': 2, 'price': 10.0}]",[],"['Scribe', 'Jeweler']"
Bronze Ingot,117,9,7371,819,"[{'ing_name': 'Copper Ore', 'count': 10, 'price': 44.0}]","[{'ing_name': 'Lump of Tin', 'count': 1, 'price': 8}]","['Artificer', 'Weaponsmith', 'Scribe', 'Huntsman', 'Armorsmith']"
Ancient Wood Plank,494,2,2316,1158,"[{'ing_name': 'Ancient Wood Log', 'count': 3, 'price': 139.0}]",[],"['Artificer', 'Huntsman', 'Scribe', 'Weaponsmith']"
Bolt of Cotton,74,2,1468,734,"[{'ing_name': 'Cotton Scrap', 'count': 2, 'price': 30.0}]",[],"['Armorsmith', 'Leatherworker', 'Scribe', 'Tailor']"
Bolt of Wool,448,0,0,726,"[{'ing_name': 'Wool Scrap', 'count': 2, 'price': 190.0}]",[],"['Armorsmith', 'Leatherworker', 'Scribe', 'Tailor']"


In [2]:

import pandas as pd 

df = pd.read_json("database/refinement_recipes.json")
df = df.convert_dtypes()

output_df = pd.DataFrame()

def get_crafting_profit(output_item_name, single_resource_price, refined_material_sell_price):
    global output_df

    # this value corresponds to cost of single resource needed to craft refined material
    tp_ingredient_cost = single_resource_price
    tp_ingredient_cost *= df[df['output_item_name'] == output_item_name]['tp_ingredients'].values[0][0]['count']

    vendor_ingredient_cost = 0
    for i in df[df['output_item_name'] == output_item_name]['vendor_ingredients'].values[0]:
        vendor_ingredient_cost += i['price'] * i['count']

    crafting_price = (tp_ingredient_cost + vendor_ingredient_cost)/df[df['output_item_name'] == output_item_name]['output_item_count'].values[0]
    sell_prize_minus_taxes = refined_material_sell_price - (refined_material_sell_price * 0.05) - (refined_material_sell_price * 0.1)
    profit = sell_prize_minus_taxes - crafting_price

    tmp_dict = {
        'output_item_name' : output_item_name,
        'single_resource_price': single_resource_price,
        'refined_material_sell_price': refined_material_sell_price,
        'refining_profit': profit,
        'crafting_price': crafting_price
    }

    output_df = output_df.append(tmp_dict, ignore_index=True)

get_crafting_profit('Steel Ingot', 125, 517)
get_crafting_profit('Platinum Ingot', 128, 0)
get_crafting_profit('Bronze Ingot', 43, 116)
get_crafting_profit('Gold Ingot', 37, 88)
get_crafting_profit('Darksteel Ingot', 163, 459)

get_crafting_profit('Bolt of Silk', 17, 51)
get_crafting_profit('Bolt of Wool', 160, 407)
get_crafting_profit('Bolt of Linen', 95, 238)
get_crafting_profit('Bolt of Jute', 58, 140)
get_crafting_profit('Bolt of Cotton', 31, 87)

get_crafting_profit('Cured Thin Leather Square', 58, 144)
get_crafting_profit('Cured Thick Leather Square', 69, 360)
get_crafting_profit('Cured Rugged Leather Square', 246, 632)

get_crafting_profit('Hard Wood Plank', 139, 506)
get_crafting_profit('Elder Wood Plank', 61, 225)


with pd.option_context('display.max_rows', None, 'display.max_columns', None):
	display(output_df[[
        'output_item_name',
        'single_resource_price',
        'crafting_price',
        'refined_material_sell_price',
        'refining_profit'
    ]]

	# .sort_values(by='crafting_profit', ascending=False)

	# .style.hide_index()
	)

,output_item_name,single_resource_price,crafting_price,refined_material_sell_price,refining_profit
0,Steel Ingot,125.0,391.0,517.0,48.45
1,Platinum Ingot,128.0,256.0,0.0,-256.00
2,Bronze Ingot,43.0,87.6,116.0,11.00
3,Gold Ingot,37.0,74.0,88.0,0.80
4,Darksteel Ingot,163.0,374.0,459.0,16.15
5,Bolt of Silk,17.0,34.0,51.0,9.35
6,Bolt of Wool,160.0,320.0,407.0,25.95
7,Bolt of Linen,95.0,190.0,238.0,12.30
8,Bolt of Jute,58.0,116.0,140.0,3.00
9,Bolt of Cotton,31.0,62.0,87.0,11.95
